## Convolution

### 1D Convolution

In [1]:
def conv1d(inp, kernel):
    padding = len(kernel) // 2
    padded_inp = [0] * padding + inp + [0] * padding
    res = []
    
    for i in range(padding, len(inp) + padding):
        out = 0
        for j in range(len(kernel)):
            out += kernel[j] * padded_inp[i - padding + j]
        res.append(out)
    
    return res

In [2]:
signal = [1., 2., 3., 4., 5., 6.]
kernel = [1., 0., -1.]
output = conv1d(signal, kernel)
output

[-2.0, -2.0, -2.0, -2.0, -2.0, 5.0]

### Varify result with Pytorch

In [3]:
import torch
from torch import nn

In [4]:
conv = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, padding=1, bias=False)
with torch.no_grad():
    conv.weight[:] = torch.tensor([[kernel]])  # shape: (out_channels, in_channels, kernel_size)

signal = torch.tensor([[ signal ]])  # shape: (batch=1, channels=1, length=3)
output = conv(signal)
output

tensor([[[-2., -2., -2., -2., -2.,  5.]]], grad_fn=<ConvolutionBackward0>)

### 2D/3D Convolution

In [26]:
def conv3d(image, kernel):
    image_channels, image_height, image_width = image.shape
    kernel_channels, kernel_height, kernel_width = kernel.shape
    
    assert image_channels == kernel_channels, "channel mismatch"
    
    pad_h = (kernel_height - 1) // 2
    pad_w = (kernel_width - 1) // 2
    
    padded_image = np.pad(image, ((0,0), (pad_h, pad_h), (pad_w, pad_w)), mode='constant')
    
    output = np.zeros((image_height, image_width))
    
    for c in range(image_channels):
        for i in range(image_height):
            for j in range(image_width):
                img = padded_image[c, i:i+kernel_height, j:j+kernel_width]
                output[i,j] += np.sum(img * kernel[c])
    
    return output

In [28]:
image = np.random.rand(3, 64, 64)               # RGB image
kernel = np.random.rand(3, 3, 3)                # 3x3 kernel for each channel
out = conv3d(image, kernel)            # Output shape: (64, 64)
out.shape

(64, 64)

In [31]:
# Convert inputs to PyTorch tensors
image = torch.tensor(image).unsqueeze(0)         # (1, C, H, W)
kernel = torch.tensor(kernel).unsqueeze(0)       # (1, C, Kh, Kw)

# Set up conv2d with 1 output channel, no bias
conv = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, padding=1, bias=False)
conv.weight.data = kernel  # Set weights directly

# Apply convolution
output_pt = conv(image).squeeze().detach().numpy()  # remove batch and channel dims
output_pt.shape

(64, 64)

In [32]:
from fastcore.test import test_close

### Check if the output from our function is same as the output from the nn.Conv2D

In [33]:
test_close(out, output_pt, eps=1e-5)